In [1]:
import keras
from keras.layers import Conv1D
import numpy as np

Using TensorFlow backend.


In [2]:
import data_loader as dl

In [3]:
data = dl.DataLoader()

In [4]:
X, Y, m = data.load()


In [5]:
from keras.preprocessing.sequence import pad_sequences
len(X[0][0])

12

In [6]:
X_pad = pad_sequences(X, maxlen=m, padding='post')
Y_pad = pad_sequences(Y, maxlen=m, padding='post')

In [7]:
sample_weights = np.ones((273, m))
for i in xrange(273):
    for j in xrange(m):
        if (X_pad[i][j] == np.zeros(12)).all():
            sample_weights[i][j] = 0

In [8]:
from keras.models import Sequential

In [9]:
model = Sequential()

In [10]:
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Dropout
from keras.layers.wrappers import TimeDistributed

In [11]:
from keras.layers import Embedding

In [12]:
mask = np.zeros(12)
model.add(Masking(mask_value=mask, input_shape=(m, 12)))
model.add(Conv1D(100, return_sequences=True))
model.add(TimeDistributed(Dense(12, activation="softmax")))

TypeError: __init__() takes at least 3 arguments (2 given)

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              sample_weight_mode='temporal')

In [ ]:
X_train, X_test = X_pad[:136, :], X_pad[136:, :]

In [ ]:
Y_train, Y_test = Y_pad[:136, :], Y_pad[136:, :]

In [ ]:
sample_weights_train, sample_weights_test = sample_weights[:136, :], sample_weights[136:, :]

In [ ]:
# # for custom metrics
import numpy as K

# def weighted_accuracy(y_true, y_pred):
#     score_array *= sample_weights
# score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))

def weighted_accuracy(y_true, y_pred):
    # Only for testing
#     score_array = K.equal(K.argmax(y_true, axis=-1),
#                           K.argmax(y_pred, axis=-1))
#     score_array *= weights
#     score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))
#     return K.mean(score_array)
    total = 0.0
    count = 0.0
    for i, y_i in enumerate(y_true):
        for j, y_ij in enumerate(y_i):
            if sum(y_ij) > 0:
                total += y_ij[y_pred[i][j]]
                count += 1
    return total / count

In [ ]:
model.fit(X_train, Y_train, batch_size=136, nb_epoch=100, sample_weight=sample_weights_train)

In [ ]:
Y_prediction = model.predict_classes(X_test, batch_size=5)

In [ ]:
# Y_prediction = to_categorical(Y_prediction)
# weighted_accuracy(Y_test, Y_prediction, sample_weights_test)
weighted_accuracy(Y_test, Y_prediction)

In [ ]:
model.evaluate(X_test, Y_test, batch_size=137, sample_weight=sample_weights_test)

In [ ]:
Y_prediction[0]

In [ ]:
Y_prediction.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
len(model.get_weights())

In [ ]:
for w in model.get_weights():
    if len(w.shape) > 1:
        plt.figure()
        plt.imshow(w)

In [ ]:
count = 0
for y in Y_prediction:
    if sum(y) > 0:
        count += 1
        print sum(y)
print count